# Метод k-ближайших соседей.

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
import sys
import random
from collections import Counter

*Разбиваем датасет на две части: для теста и для данных проверки*

In [2]:
def split_dataset(data):
    len_mass = len(data.data) # длина массива данных
    mass_sort = [i for i in range(len_mass)] # массив индексов
    random.shuffle(mass_sort) # Рандомно сортируем индексы

    len_determine = int(len_mass * 0.7)
    # list_determine = [len_mass - len_determine, len_determine]
    datasets_learn = [[data.data[i], data.target[i]] for i in mass_sort[:len_determine]] # Первые 70 процентов добавляем для обучение,
    datasets_test = [[data.data[i], data.target[i]] for i in mass_sort[len_determine:]] # остальные для теста.
    return datasets_learn, datasets_test


*Нормализуем данные*
**ПЕРЕПИСАТЬ**

In [3]:
def standartization(data):
    # Находим макс и мин эл.
    min_elem = min(data, key = lambda a: min(a[0]))[0][-1]
    max_elem = max(data, key = lambda a: max(a[0]))[0][0]

    print(min_elem, max_elem)
    difference = max_elem - min_elem

    # Меняем значения на нормализованные.
    for i in range(len(data)):
        elem_support = 0
        for j in range(len(data[i][0])):
            data[i][0][j] = (data[i][0][j] - min_elem) / difference


*Метрики*

In [4]:
# Вход el1 = [el1, el2, ... , eln]
def l1(el1, el2):
    return sum([abs(k - v) for k, v in list(zip(el1, el2))])

def l2(el1, el2):
    return sum([(k - v)**2 for k, v in list(zip(el1, el2))]) ** 0.5

def l_infinity(el1, el2):
    max_len = 0
    for k, v in list(zip(el1, el2)):
        if abs(k- v) > max_len:
            max_len = abs(k - v)
    return max([abs(k - v) for k, v in list(zip(el1, el2))])

*Метод k-ближайших соседей*

In [5]:
def choice_class(mass_n):
    count = Counter(mass_n)
    return max(count, key = count.get)


def k_nearest_neightbors(data_learn, data_test, metric = l1, n = 4):
    rez = []
    for i in data_test:
        nearest_neighbors_list = []
        mass = []
        for j in data_learn:
            mass.append([metric(i[0], j[0]), j[1]])
        mass.sort(key = lambda a: a[0])
        for z in mass[:n]:
            nearest_neighbors_list.append(z[1])
        rez.append(choice_class(nearest_neighbors_list))
    return rez

In [6]:
def check_forecast(data_test, forecast):
    target_rez = len(data_test)
    forecast_rez = 0
    for i in range(len(data_test)):
        if data_test[i][1] == forecast[i]:
            forecast_rez += 1
    return forecast_rez / target_rez


In [7]:
def main():
    data = datasets.load_wine()

    learn, test = split_dataset(data)

    standartization(learn)
    standartization(test)
    

    # 
    rez_l1 = k_nearest_neightbors(learn, test, l1, 5)
    rez_l2 = k_nearest_neightbors(learn, test, l2, 5)
    rez_l_infinity = k_nearest_neightbors(learn, test, l_infinity, 5)

    answer1 = check_forecast(test, rez_l1)
    answer2 = check_forecast(test, rez_l2)
    answer3 = check_forecast(test, rez_l_infinity)

    print(f"Обучение с метрикой l1 для 5 ближайших соседей: {answer1}")
    print(f"Обучение с метрикой l2 для 5 ближайших соседей: {answer2}")
    print(f"Обучение с метрикой l_infinity для 5 ближайших соседей: {answer3}")

In [8]:
if __name__ == "__main__":
    main()

886.0 14.19
845.0 14.2
Обучение с метрикой l1 для 5 ближайших соседей: 0.6851851851851852
Обучение с метрикой l2 для 5 ближайших соседей: 0.6851851851851852
Обучение с метрикой l_infinity для 5 ближайших соседей: 0.6666666666666666
